In [5]:
#Библиотеки
import numpy as np
from numba import cuda
from time import time
import math

In [4]:
!lscpu |grep 'Model name'
!nvidia-smi

Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Mon Nov  2 20:02:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    29W /  70W |    233MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                    

In [6]:
#Построчное вычисление произведения матриц
def simple_dot_cpu(matrix1, matrix2):
  N = len(matrix1)
  matrix_res = np.zeros((N,N))
  start = time()
  for i in range(N):
    for j in range(N):
      for k in range(N):
        matrix_res[i, j] += matrix1[i,k] * matrix2[k,j]
  return matrix_res, time()-start

In [7]:
#встроенная функция для подсчета произведения
def numpy_dot_cpu(matrix1, matrix2):
  start = time()
  matrix_res = np.dot(matrix1, matrix2)
  return matrix_res, time() - start

In [8]:
#подключается ядро cuda, вычисляется один элемент
@cuda.jit
def simple_dot_gpu(matrix1, matrix2, matrix_res):
    i, j = cuda.grid(2)
    if i < matrix_res.shape[0] and j < matrix_res.shape[1]:
      temp = 0
      for k in range(matrix1.shape[1]):
        temp += matrix1[i, k] * matrix2[k, j]
      matrix_res[i, j] = temp

In [9]:
def gpu_dot_exec(matrix1, matrix2, matrix_res, N):
  #нити в блоке 
  tread_number_block = 32
  #создаем копии матриц и результата на gpu
  matrix1_global = cuda.to_device(matrix1)
  matrix2_global = cuda.to_device(matrix2)
  matrix_res_global = cuda.device_array((N, N))
    
  #создание сетки
  threadsperblock = (tread_number_block, tread_number_block)
  blockspergrid_x = int(math.ceil(matrix1.shape[0] / threadsperblock[1]))
  blockspergrid_y = int(math.ceil(matrix2.shape[1] / threadsperblock[0]))
  blockspergrid = (blockspergrid_x, blockspergrid_y)

  start = time()
  #вызываем функцию на сетке
  simple_dot_gpu[blockspergrid, threadsperblock](matrix1_global, matrix2_global, matrix_res_global)
  gpu_time = time() - start
  matrix_res_gpu = matrix_res_global.copy_to_host() 
  return matrix_res_gpu, gpu_time

In [10]:
def one_test(N):  
  gpu_time = 0
  cpu_time = 0
  matrix1 = np.random.randint(0, 10, (N, N))
  matrix1 = matrix1.astype(np.float64)
  matrix2 = np.random.randint(0, 10, (N, N))
  matrix2 = matrix2.astype(np.float64)
  matrix_res = np.zeros((N, N))
  matrix_res = matrix_res.astype(np.float64)
    
  matrix_res_gpu, gpu_time = gpu_dot_exec(matrix1, matrix2, matrix_res, N)
  start = time()
  matrix_res_cpu = simple_dot_cpu(matrix1, matrix2)
  cpu_time += time() - start

  print('Размер матрицы N =', N)
  print('Время вычисления на CPU =',cpu_time)
  print('Время вычисления на GPU =',gpu_time)
  print('Ускорение = ',cpu_time/gpu_time )
  return cpu_time/gpu_time, matrix_res_cpu, matrix_res_gpu

In [14]:
#с помощью данной функции проверяется корректность перемножения
def check_correctly_dot(N):
  matrix1 = np.random.randint(0, 10, (N, N))
  matrix1 = matrix1.astype(np.float64)
  matrix2 = np.random.randint(0, 10, (N, N))
  matrix2 = matrix2.astype(np.float64)
  matrix_res = np.zeros((N, N))
  matrix_res = matrix_res.astype(np.float64)
  matrix_res_real = numpy_dot_cpu(matrix1,matrix2)[0]
  matrix_res_cpu = simple_dot_cpu(matrix1,matrix2)[0]
  matrix_res_gpu = gpu_dot_exec(matrix1, matrix2, matrix_res, N)[0]
  if np.array_equal(matrix_res_real, matrix_res_cpu):
    print('Перемножение на CPU корректно')
  else:
    print('Перемножение на CPU НЕ корректно')
  if np.array_equal(matrix_res_real, matrix_res_gpu):
    print('Перемножение на GPU корректно')
  else:
    print('Перемножение на GPU НЕ корректно')

In [15]:
check_correctly_dot(128)
result = np.zeros((5,2), dtype = np.float64)
result[0][0], result[0][1] = 128, one_test(128)[0]
result[1][0], result[1][1] = 256, one_test(256)[0]
result[2][0], result[2][1] = 512, one_test(512)[0]
result[3][0], result[3][1] = 1024, one_test(1024)[0]
result[4][0], result[4][1] = 2048, one_test(2048)[0]

Перемножение на CPU корректно
Перемножение на GPU корректно
Размер матрицы N = 128
Время вычисления на CPU = 1.4991044998168945
Время вычисления на GPU = 0.00028705596923828125
Ускорение =  5222.34219269103
Размер матрицы N = 256
Время вычисления на CPU = 12.083741903305054
Время вычисления на GPU = 0.0003867149353027344
Ускорение =  31247.15598027127
Размер матрицы N = 512
Время вычисления на CPU = 101.23723149299622
Время вычисления на GPU = 0.0004267692565917969
Ускорение =  237217.72346368714
Размер матрицы N = 1024
Время вычисления на CPU = 809.2119972705841
Время вычисления на GPU = 0.00045371055603027344
Ускорение =  1783542.3631108776
Размер матрицы N = 2048
Время вычисления на CPU = 6453.193475246429
Время вычисления на GPU = 0.0004534721374511719
Ускорение =  14230628.394321766
